In [1]:
from mindspore import context
from mindspore.nn import Adam
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

## 分子系统的复制与扩展

在分子对象的最基础的类Molecule中，支持了reduplicate和copy两种复制的模式，其实是同样的操作，不同点在于，reduplicate复制之后会默认把新生成的分子添加到此前定义的分子体系中去。而copy则是创建了一个新的对象，需要通过Molecule内置的append方法，才能把copy出来的对象添加到原始的分子系统中去。

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

## 优化器与积分器

在MindSponge与传统深度学习框架的架构图示意中，我们可以发现分子动力学模拟过程中的积分器与深度学习中的优化器，本质上其实是一回事，都需要获取到gradient然后再去对系统进行更新。而如果在分子动力学模拟的过程中直接使用传统优化器如GradientDescent和Adam等来进行能量最小化，也是一个很重要的过程。在执行动力学模拟过程之前，如果不对系统进行能量最小化，将很有可能在运行过程中出现梯度爆炸或者梯度消失的问题，会为系统的正常运行带来众多的不确定性。

![](../../docs/img.png)

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
md = Sponge(system, potential, opt)

In [8]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b03.h5md', save_freq=10)

In [9]:
md.run(1000, callbacks=[run_info, cb_h5md])

Step: 0, E_pot: -9490.139, 
Step: 10, E_pot: -10437.403, 
Step: 20, E_pot: -10481.724, 
Step: 30, E_pot: -10501.464, 
Step: 40, E_pot: -10531.801, 
Step: 50, E_pot: -10552.642, 
Step: 60, E_pot: -10576.384, 
Step: 70, E_pot: -10598.561, 
Step: 80, E_pot: -10617.948, 
Step: 90, E_pot: -10635.134, 
Step: 100, E_pot: -10650.692, 
Step: 110, E_pot: -10663.982, 
Step: 120, E_pot: -10675.984, 
Step: 130, E_pot: -10687.878, 
Step: 140, E_pot: -10700.114, 
Step: 150, E_pot: -10712.279, 
Step: 160, E_pot: -10723.851, 
Step: 170, E_pot: -10734.402, 
Step: 180, E_pot: -10743.937, 
Step: 190, E_pot: -10752.699, 
Step: 200, E_pot: -10760.898, 
Step: 210, E_pot: -10768.495, 
Step: 220, E_pot: -10775.1455, 
Step: 230, E_pot: -10780.383, 
Step: 240, E_pot: -10784.105, 
Step: 250, E_pot: -10786.751, 
Step: 260, E_pot: -10788.78, 
Step: 270, E_pot: -10790.408, 
Step: 280, E_pot: -10791.747, 
Step: 290, E_pot: -10792.88, 
Step: 300, E_pot: -10793.861, 
Step: 310, E_pot: -10794.742, 
Step: 320, E_pot: -10

![](../../docs/tutorial_b03.gif)